usar o kenel yolo (home)

In [24]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tools import *
import glob
import os
import cv2
import numpy as np
from sahi.slicing import slice_image
import matplotlib.pyplot as plt

print(tf.__version__)
print(tf.config.list_physical_devices())


2.20.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [25]:
model = load_model("../model/Colab_23-12-2025.keras",
                   custom_objects={"dice_loss": dice_loss,
                                   "dice_coefficient": dice_coefficient})

In [26]:
input_path  = r"../Dataset\Validacao\Exp"
output_dir  = r"../Dataset\Validacao\predict"

filename = os.path.basename(input_path)
imgs_dir = sorted(glob.glob(os.path.join(input_path, '*.*')))[:]

In [27]:

for img_path in imgs_dir:
    image = cv2.imread(img_path)
    H, W = image.shape[:2]

    sliced = slice_image(
        image=image,
        slice_height=128,
        slice_width=128,
        overlap_height_ratio=0.4,
        overlap_width_ratio=0.4,
        auto_slice_resolution=False
    )

    mask_acc = np.zeros((H, W), dtype=np.float32)
    weight   = np.zeros((H, W), dtype=np.float32)
 
    tiles = []
    for tile in sliced.images:
        img_gray = cv2.cvtColor(tile, cv2.COLOR_RGB2GRAY)
        img_norm = normalized_image(img_gray)
        tiles.append(img_norm)

    tiles = np.stack(tiles, axis=0)[..., np.newaxis]
    # shape: (N, 128, 128, 1)

    
    mask_preds = model.predict(tiles, batch_size=16, verbose=0)
    mask_preds = mask_preds[..., 0]  # (N, 128, 128)
    mask_preds = (mask_preds > 0.3).astype(np.float32)
    

    for mask_tile, (x0, y0) in zip(mask_preds, sliced.starting_pixels):
        h, w = mask_tile.shape
        mask_acc[y0:y0+h, x0:x0+w] += mask_tile
        weight[y0:y0+h, x0:x0+w]   += 1

    mask_full = mask_acc / np.maximum(weight, 1)
    
    filename = os.path.basename(img_path)
    output_path = os.path.join(output_dir, filename)

    mask_to_save = (mask_full * 255).astype(np.uint8)
    cv2.imwrite(output_path, mask_to_save)